In [5]:
import json

with open("/home/dgmastertemple/Documents/GitHub/create-audio-bible/projects/Ephesians - Tim Conway/json/readings.json", "r") as f:
    data: dict = json.load(f)

In [4]:

for ref, finds in data.items()[:1]:
    print(ref)

TypeError: 'dict_items' object is not subscriptable